# California Housing data from 1990 US Census

In this lab we will rely on [Pandas](https://pandas.pydata.org/) and [Seaborn](http://seaborn.pydata.org/) to inspect and visualize data.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import time
import jax.numpy as jnp
import jax

Load the dataset (if you are using Google Colab, it is already in the `sample_data` folder!).

In [ ]:
data = pd.read_csv('sample_data/california_housing_train.csv')

## Data inspection

Display some basic information.

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.describe()

We are interested in predicting the field `median_house_value`. Plot its distribution.

In [ ]:
sns.distplot(data['median_house_value'])

It looks like the distribution tail has been truncated. Get rid of it to ease the prediction.

In [ ]:
data = data[data['median_house_value'] < 500001]
sns.distplot(data['median_house_value'])

Use a scatterplot to visualize the geograhical distribution of the houses.

In [ ]:
sns.scatterplot(data = data, x='longitude' ,y='latitude', hue='median_house_value')

Look for linear correlations among features.

In [ ]:
data.corr()

In [ ]:
sns.heatmap(data.corr(), annot = True, cmap = 'vlag_r', vmin = -1, vmax = 1)

In [ ]:
sns.scatterplot(data = data, x='latitude' ,y='median_house_value')

## Data normalization

Apply an affine transformation to the data, so that each feature has zero mean and unitary standard deviation.

In [ ]:
data_mean = data.mean()
data_std = data.std()
data_normalized = (data - data_mean) / data_std

data_normalized.describe()

In [ ]:
_, ax = plt.subplots(figsize=(16,6))
sns.violinplot(data = data_normalized, ax = ax)

## Train-validation split

Shuffle the data (**hint:** use the [np.random.shuffle](https://numpy.org/doc/stable/reference/random/generated/numpy.random.shuffle.html) function) and split the data as follows:
- put 80% in the train dataset
- put 20% in the validation dataset

In [ ]:
np.random.seed(0) # for reproducibility
data_normalized_np = data_normalized.to_numpy()
np.random.shuffle(data_normalized_np)

fraction_validation = 0.2
num_train = int(data_normalized_np.shape[0] * (1 - fraction_validation))
x_train = data_normalized_np[:num_train,:-1]
y_train = data_normalized_np[:num_train,-1:]
x_valid = data_normalized_np[num_train:,:-1]
y_valid = data_normalized_np[num_train:,-1:]

## ANN setup

Write a function `initialize_params` that, given the input `layers_size = [n1, n2, ..., nL]`, generates the parameters associated with an ANN, having as many layers as the number of elements of `layers_size`, with as many neurons as `n1`, `n2`, etc.

To initialize the parameters, employ the following strategy:
- Inizialize the biases with zero value.
- Inizialize the weights sampling from a Gaussian distribution with zero mean and with standard deviation 
$$
\sqrt{\frac{2}{n + m}}
$$
where $n$ and $m$ are the number of input and output neurons of the corresponding weights matrix (this is known as "Glorot Normal" or "Xavier Normal", see [Glorot, Bengio 2010](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf)).

Other popular initializations strategies are:
- Gaussian distribution with zero mean and with standard deviation (for some constant $K$)
$$
\frac{K}{\sqrt{n}}
$$
- Uniform distribution
$$
\left[-\sqrt{\frac{1}{n}}, \sqrt{\frac{1}{n}}\right]
$$
- Uniform distribution (this is known as "Glorot Uniform" or "Xavier Uniform")
$$
\left[-\sqrt{\frac{6}{n + m}}, \sqrt{\frac{6}{n + m}}\right]
$$

In [ ]:
def initialize_params(layers_size):
  np.random.seed(0) # for reproducibility
  params = list()
  for i in range(len(layers_size) - 1):
    W = np.random.randn(layers_size[i+1], layers_size[i]) * np.sqrt(2 / (layers_size[i+1] + layers_size[i]))
    b = np.zeros((layers_size[i+1], 1))
    params.append(W)
    params.append(b)
  return params

params = initialize_params([8, 5, 5, 1])
params

Implement a generic feedforward ANN with a function `y = ANN(x, params)', using $\tanh$ as activation function.

By convention, both the input and the output have:
- 1 sample per row
- 1 feature per column

In [ ]:
activation = jnp.tanh
# activation = lambda x: jnp.maximum(0.0, x)

def ANN(x, params):
  layer = x.T
  num_layers = int(len(params) / 2 + 1)
  weights = params[0::2]
  biases = params[1::2]
  for i in range(num_layers - 1):
    layer = jnp.dot(weights[i], layer) - biases[i]
    if i < num_layers - 2:
      layer = activation(layer)
  return layer.T

ANN(x_train, params)
x_train

Implement the quadratic (MSE) loss function `L = loss(x, y, params)`, defined as:

$$
\mathcal{L}(\mathbf{x}, \mathbf{y}, \boldsymbol{\theta}) = \frac{1}{m} \sum_{i=1}^m (y_i - \mathrm{ANN}(x_i, \boldsymbol{\theta}))^2
$$

where $m$ is the number of samples in $\mathbf{x}$, $\mathbf{y}$ and $\boldsymbol{\theta}$ are the ANN parameters.

In [ ]:
def loss(x, y, params):
  error =  ANN(x, params) - y
  return jnp.mean(error * error)

Test your code, by generating the parameters associated with an ANN with two hidden layers with 5 neurons each and by computing the associated loss.

In [ ]:
params = initialize_params([8, 5, 5, 1])
loss(x_train, y_train, params)

## Training

### Gradient Descent

Implement the GD method:
$$
\begin{split}
& \boldsymbol{\theta}^{(0)} \text{given} \\
& \text{for } k = 0, 1, \dots , n_{\text{epochs}} - 1\\
& \qquad \mathbf{g}^{(k)} = \frac{1}{N} \sum_{i=1}^N \nabla_{\boldsymbol{\theta}} \mathcal{L}(x_i, y_i, \boldsymbol{\theta}^{(k)}) \\
& \qquad \boldsymbol{\theta}^{(k+1)} = \boldsymbol{\theta}^{(k)} - \lambda \mathbf{g}^{(k)}
\end{split}
$$
where $N$ is the number of training samples. At each iteration, append the current cost to the list `history`.

Train an ANN with two hidden layers with 20 neurons each. 
Try to (manually) optimize the training hyperparameters.

During training, store the MSE error obtained on the train and validation sets in two lists, respectively called `history_train` and `history_valid`. Finally, plot the erros trend and diplay the final values of the errors.



Hints: 
- Use `jax.jit` to speedup the evaluation of the loss and of the gradients.
- Use `tqdm` to show a nice progress bar:

In [ ]:
from tqdm.notebook import tqdm
for i in tqdm(range(100)):
  # do something ...
  time.sleep(0.02) # only for testing

In [ ]:
# Hyperparameters
layers_size = [8, 20, 20, 1]
# Training options
num_epochs = 2000
learning_rate = 1e-1
########################################

params = initialize_params(layers_size)

grad = jax.jit(jax.grad(loss, argnums = 2))
loss_jit = jax.jit(loss)
grad_jit = jax.jit(grad)

n_samples = x_train.shape[0]

history_train = list()
history_valid = list()
history_train.append(loss_jit(x_train, y_train, params))
history_valid.append(loss_jit(x_valid, y_valid, params))

t0 = time.time()
for epoch in tqdm(range(num_epochs)):
  grads = grad_jit(x_train, y_train, params)
  
  for i in range(len(params)):
    params[i] -= learning_rate * grads[i]

  history_train.append(loss_jit(x_train, y_train, params))
  history_valid.append(loss_jit(x_valid, y_valid, params))

print('elapsed time: %f s' % (time.time() - t0))
print('loss train     : %1.3e' % history_train[-1])
print('loss validation: %1.3e' % history_valid[-1])

fig, axs = plt.subplots(1, figsize=(16,8))
axs.loglog(history_train, label = 'train')
axs.loglog(history_valid, label = 'validation')

### Stochastic Gradient Descent

Implement the SGD method:
$$
\begin{split}
& \boldsymbol{\theta}^{(0)} \text{given} \\
& \text{for } k = 0, 1, \dots , n_{\text{epochs}} - 1\\
& \qquad \mathbf{g}^{(k)} = \frac{1}{|I_k|} \sum_{i \in I_k} \nabla_{\boldsymbol{\theta}} \mathcal{L}(x_i, y_i, \boldsymbol{\theta}^{(k)}) \\
& \qquad \boldsymbol{\theta}^{(k+1)} = \boldsymbol{\theta}^{(k)} - \lambda_k \mathbf{g}^{(k)}
\end{split}
$$
where $I_k$ is the current minibatch. To select it, use the function [np.random.choice](https://docs.scipy.org/doc//numpy-1.10.4/reference/generated/numpy.random.choice.html) with replacement.

Consider a linear decay of the learning rate:
$$
\lambda_k = \max\left(\lambda_{\textnormal{min}}, \lambda_{\textnormal{max}} \left(1 - \frac{k}{K}\right)\right)
$$

Test different choices of batch size and try to optimize the learning rate decay strategy.

In [ ]:
# Hyperparameters
layers_size = [8, 20, 20, 1]
# Training options
num_epochs = 2000
learning_rate_max = 2e-1
learning_rate_min = 5e-2
learning_rate_decay = 2000
batch_size = 1000
########################################

params = initialize_params(layers_size)

grad = jax.jit(jax.grad(loss, argnums = 2))
loss_jit = jax.jit(loss)
grad_jit = jax.jit(grad)

n_samples = x_train.shape[0]

history_train = list()
history_valid = list()
history_train.append(loss_jit(x_train, y_train, params))
history_valid.append(loss_jit(x_valid, y_valid, params))

t0 = time.time()
for epoch in tqdm(range(num_epochs)):
  learning_rate = max(learning_rate_min, learning_rate_max * (1 - epoch/learning_rate_decay))
  idxs = np.random.choice(n_samples, batch_size)
  grads = grad_jit(x_train[idxs,:], y_train[idxs,:], params)
  
  for i in range(len(params)):
    params[i] -= learning_rate * grads[i]

  history_train.append(loss_jit(x_train, y_train, params))
  history_valid.append(loss_jit(x_valid, y_valid, params))

print('elapsed time: %f s' % (time.time() - t0))
print('loss train     : %1.3e' % history_train[-1])
print('loss validation: %1.3e' % history_valid[-1])

fig, axs = plt.subplots(1, figsize=(16,8))
axs.loglog(history_train, label = 'train')
axs.loglog(history_valid, label = 'validation')

## Testing

Load the test dataset `sample_data/california_housing_test.csv` and use the trained model to predict the house prices of the dataset.

Compare predicted prices with actual prices by means of a scatterplot.

Finally, compute the RMSE (root mean square error).

In [ ]:
data_test = pd.read_csv('sample_data/california_housing_test.csv')
data_test = data_test[data_test['median_house_value'] < 500001]
data_test_normalized = (data_test - data.mean()) / data.std()
x_test = data_test_normalized.drop('median_house_value',axis=1).to_numpy()
Y_test = data_test['median_house_value'].to_numpy()[:,None]

y_predicted = ANN(x_test, params)
Y_predicted = (y_predicted * data['median_house_value'].std()) + data['median_house_value'].mean()

In [ ]:
test = pd.DataFrame({'predicted': Y_predicted[:,0],'actual': Y_test[:,0]})
fig = sns.jointplot(data=test, x='actual', y='predicted')
fig.ax_joint.plot([0, 500000], [0, 500000.], 'r')

In [ ]:
error = Y_test - Y_predicted
RMSE = jnp.sqrt(jnp.mean(error * error))
print('RMSE: %0.2f k$' % (RMSE * 1e-3))